In [ ]:
# define rooms and items

couch = {
    "name": "couch",
    "type": "furniture",
}
piano = {
    "name": "piano",
    "type": "furniture",
}
queen_bed = {
    "name": "queen bed",
    "type": "furniture"
}
double_bed =  {
    "name": "double bed",
    "type": "furniture"
}
dresser = {
    "name": "dresser",
    "type": "furniture"
}
dining_table = {
    "name": "dining table",
    "type": "furniture"
}
# ---------------------------------------------------------------------------------------------------------------------------------------------------------

door_a = {
    "name": "door a",
    "type": "door",
    "locked": True
}

door_b = {
    "name": "door b",
    "type": "door",
    "locked": True
}
door_c = {
    "name": "door c",
    "type": "door",
    "locked": True
}
door_d = {
    "name": "door d",
    "type": "door",
    "locked": True
}

# ---------------------------------------------------------------------------------------------------------------------------------------------------------

key_a = {
    "name": "key for door a",
    "type": "key",
    "target": door_a,
}
key_b = {
    "name": "key for door b",
    "type": "key",
    "target": door_b,
}

key_c = {
    "name": "key for door c",
    "type": "key",
    "target": door_c,
}

key_d = {
    "name": "key for door d",
    "type": "key",
    "target": door_d,
}



game_room = {
    "name": "game room",
    "type": "room",
}
bedroom_1 = {
    "name": "bedroom 1",
    "type": "room",
}
bedroom_2 = {
    "name": "bedroom 2",
    "type": "room",
}
living_room = {
    "name": "living room",
    "type": "room",
}

outside = {
  "name": "outside"
}

all_rooms = [game_room, bedroom_1, bedroom_2, living_room, outside]

all_doors = [door_a, door_b, door_c, door_d]

# define which items/rooms are related

# object_relations = {
#     "game room": [couch, piano, door_a],
#     "piano": [key_a],
#     "outside": [door_a],
#     "door a": [game_room, outside]
# }

object_relations = {
    "game room": [couch, piano, door_a],
    "piano": [key_a],
    "door a": [game_room, bedroom_1],
    "bedroom 1": [door_a, queen_bed, door_b, door_c],
    "queen bed": [key_b],
    "door b": [bedroom_1, bedroom_2],
    "bedroom 2": [door_b, double_bed, dresser],
    "double bed": [key_c],
    "dresser": [key_d],
    "door c": [bedroom_1, living_room],
    "living room": [dining_table, door_d],
    "door d": [living_room, outside],
    "outside": [door_d],
}

# define game state. Do not directly change this dict.
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "target_room": outside
}

In [ ]:
def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def start_game():
    """
    Start the game
    """
    print("You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!")
    play_room(game_state["current_room"])

def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either
    explore (list all items in this room) or examine an item found here.
    """
    game_state["current_room"] = room

    if(game_state["current_room"] == game_state["target_room"]):
        print("Congrats! You escaped the room!")
    else:
        print("You are now in " + room["name"])
        intended_action = input("What would you like to do? Type 'explore', 'examine', 'unlock door', or 'move room': ").strip().lower()
        if intended_action == "explore":
            explore_room(room)
        elif intended_action == "examine":
            examine_item(input("What would you like to examine? ").strip())
        elif intended_action == "unlock door":
            unlock_door(room)
        elif intended_action == "move room":
            move_room(room)
        else:
            print("Not sure what you mean. Please choose a valid action.")
        linebreak()
        play_room(room)  # Recursively call play_room to continue the game loop


def move_room(current_room):
    """
    Move to other rooms, check if doors are unlocked
    """
    print("You can see doors leading to other rooms:")
    doors_in_room = [item for item in object_relations[current_room["name"]] if item["type"] == "door"] #This line is designed to generate list of all the doors in the current room.

    # Display doors and their status
    for door in doors_in_room:
        if door["locked"]:
            print(f"{door['name']} is locked.")
        else:
            print(f"{door['name']} is unlocked.")

    linebreak()
    # Player chooses a door
    chosen_door_name = input("Which door would you like to go through? Please type the door name exactly as shown above: ").strip()

    # Check if the door exists and if its locked or unlocked
    for door in doors_in_room:
        if door["name"].lower() == chosen_door_name.lower():
            if door["locked"]:
                print(f"Sorry, {chosen_door_name} is locked. You need to unlock it first if you have the key.")
            else:
                next_room = get_next_room_of_door(door, current_room)
                print(f"Moving through {chosen_door_name}, you enter {next_room['name']}.")
                play_room(next_room)
                return  # Ends the function after moving to the next room or informing about the locked status

    print("There doesn't seem to be a door by that name here. Please check the door name and try again.") #invalid door selection

def unlock_door(room):
    """
    Action to unlock doors
    """
    door_name = input("Which door would you like to unlock? ").strip() #Ask the player to choose a door to unlock.
    for item in object_relations[room["name"]]: #search if the door is in this room
        if item["type"] == "door" and item["name"] == door_name: #is the door locked?
            if not item["locked"]: #the door is found and it is not locked (item["locked"] is False), the player is informed that the door is already unlocked, and the function exits.
                print(f"{door_name} is already unlocked.")
                return
            have_key = any(key["target"] == item for key in game_state["keys_collected"]) #this line checks if the player has the key for that door
            if have_key:
                item["locked"] = False  # Unlock the door
                print(f"You unlock {door_name} with a key you have.")
                return
            else: #When player does not have the key :(
                print(f"{door_name} is locked, and you don't have the key.")
                return
    print("The door you requested is not found in the current room.") #First ondition has not found the door in current objett_relations room

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print("You explore the room. This is " + room["name"] + ". You find " + ", ".join(items))

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if(not current_room == room):
            return room

def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None

    for item in object_relations[current_room["name"]]:
        if(item["name"] == item_name):
            output = "You examine " + item_name + ". "
            if(item["type"] == "door"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        have_key = True
                if(have_key):
                    output += "You unlock it with a key you have."
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "It is locked but you don't have the key."
            else:
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "You find " + item_found["name"] + "."
                else:
                    output += "There isn't anything interesting about it."
            print(output)
            break

    if(output is None):
        print("The item you requested is not found in the current room.")

    if(next_room and input("Do you want to go to the next room? Ener 'yes' or 'no'").strip() == 'yes'):
        play_room(next_room)
    else:
        play_room(current_room)

In [ ]:
game_state = INIT_GAME_STATE.copy()

start_game()

You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!
You are now in game room
What would you like to do? Type 'explore', 'examine', 'unlock door', or 'move room': move room
You can see doors leading to other rooms:
door a is locked.



Which door would you like to go through? Please type the door name exactly as shown above: door a
Sorry, door a is locked. You need to unlock it first if you have the key.
There doesn't seem to be a door by that name here. Please check the door name and try again.



You are now in game room


KeyboardInterrupt: Interrupted by user